# Tech Challenge - Fine-tuning para Produtos Amazon

**Objetivo**: Executar fine-tuning de um foundation model usando o dataset AmazonTitles-1.3MM para gerar descrições de produtos baseadas em títulos.

**Dataset**: Utilizaremos o arquivo `trn.json.gz` que contém títulos e descrições de produtos da Amazon.

**Modelo Escolhido**: Llama 3-8B com Unsloth para otimização de treinamento.

---

## 📋 Índice
1. [Configuração Inicial](#1-configuracao-inicial)
2. [Exploração dos Dados](#2-exploracao-dos-dados)
3. [Preparação do Dataset](#3-preparacao-do-dataset)
4. [Teste do Modelo Base](#4-teste-do-modelo-base)
5. [Fine-tuning](#5-fine-tuning)
6. [Teste do Modelo Treinado](#6-teste-do-modelo-treinado)
7. [Demonstração Interativa](#7-demonstracao-interativa)

---

## 1. Configuração Inicial

### 1.1 Montagem do Google Drive
Primeiro, vamos montar o Google Drive para acessar e salvar nossos arquivos.

In [ ]:
from google.colab import drive
import os

# Monta o Google Drive
drive.mount('/content/drive')

# Define o diretório de trabalho (usando o mesmo diretório onde está o arquivo de dados)
WORK_DIR = '/content/drive/MyDrive/FineTunning/TechChallenge03'
os.makedirs(WORK_DIR, exist_ok=True)

print(f"✅ Google Drive montado com sucesso!")
print(f"📁 Diretório de trabalho: {WORK_DIR}")

# Verifica se o diretório existe e lista os arquivos
if os.path.exists(WORK_DIR):
    files_in_dir = os.listdir(WORK_DIR)
    print(f"📋 Arquivos no diretório: {files_in_dir}")
else:
    print(f"⚠️ Diretório não existe, será criado: {WORK_DIR}")

### 1.2 Instalação das Dependências

Instalamos as bibliotecas necessárias:
- **Unsloth**: Otimização para fine-tuning eficiente
- **Transformers**: Biblioteca principal para modelos de linguagem
- **Datasets**: Para manipulação de datasets
- **TRL**: Para treinamento de modelos de linguagem

In [ ]:
# Instalação das dependências principais
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets torch

print("✅ Todas as dependências foram instaladas com sucesso!")

### 1.3 Importação das Bibliotecas e Configurações Iniciais

In [ ]:
# Imports necessários
import json
import gzip
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
import torch
from transformers import TrainingArguments, TextStreamer
from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso!")
print(f"🔥 CUDA disponível: {torch.cuda.is_available()}")
print(f"💾 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Não disponível'}")

### 1.4 Configurações do Modelo e Treinamento

In [ ]:
# Configurações principais
CONFIG = {
    # Configurações do modelo
    'model_name': "unsloth/llama-3-8b-bnb-4bit",
    'max_seq_length': 2048,
    'dtype': None,  # Será determinado automaticamente
    'load_in_4bit': True,
    
    # Configurações do dataset
    'data_file': '/content/drive/MyDrive/FineTunning/TechChallenge03/trn.json.gz',
    'sample_size': 1000000,  # Número de amostras para treinamento (pode ajustar)
    'test_size': 100000,  # Número de amostras para teste
    
    # Configurações do fine-tuning
    'lora_r': 16,
    'lora_alpha': 16,
    'lora_dropout': 0,
    'max_steps': 100,  # Ajuste conforme necessário
    'learning_rate': 2e-4,
    'batch_size': 2,
    'gradient_accumulation_steps': 4,
    
    # Caminhos - usando o mesmo diretório base
    'base_dir': '/content/drive/MyDrive/FineTunning/TechChallenge03',
    'output_dir': '/content/drive/MyDrive/FineTunning/TechChallenge03/outputs',
    'model_save_path': '/content/drive/MyDrive/FineTunning/TechChallenge03/amazon_model',
}

# Criação dos diretórios
os.makedirs(CONFIG['base_dir'], exist_ok=True)
os.makedirs(CONFIG['output_dir'], exist_ok=True)
os.makedirs(CONFIG['model_save_path'], exist_ok=True)

print("⚙️ Configurações definidas:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 2. Exploração dos Dados

### 2.1 Upload do Arquivo de Dados

Primeiro, você precisa fazer upload do arquivo `trn.json.gz` para o Colab.
Execute a célula abaixo e faça upload do arquivo quando solicitado.

In [ ]:
import os

# Define o caminho para o arquivo no Google Drive
DATA_FILE_PATH = '/content/drive/MyDrive/FineTunning/TechChallenge03/trn.json.gz'

# Verifica se o arquivo existe
if os.path.exists(DATA_FILE_PATH):
    print("✅ Arquivo trn.json.gz encontrado no Google Drive!")
    print(f"📁 Caminho: {DATA_FILE_PATH}")
    
    # Verifica o tamanho do arquivo
    file_size = os.path.getsize(DATA_FILE_PATH)
    print(f"📊 Tamanho do arquivo: {file_size / (1024*1024):.1f} MB")
else:
    print("❌ Arquivo não encontrado no caminho especificado.")
    print(f"❌ Caminho verificado: {DATA_FILE_PATH}")
    print("💡 Certifique-se de que o arquivo trn.json.gz está no diretório correto do Google Drive.")
    DATA_FILE_PATH = None

### 2.2 Carregamento dos Dados

Vamos carregar o dataset completo (ou conforme configurado) e analisar sua estrutura para entender melhor os dados com que estamos trabalhando.

In [ ]:
def load_amazon_data(file_path, sample_size=None):
    """
    Carrega os dados do arquivo JSON comprimido, extraindo apenas title e content
    
    Args:
        file_path: Caminho para o arquivo trn.json.gz
        sample_size: Número de amostras a carregar (None para carregar tudo)
    
    Returns:
        Lista de dicionários com apenas os campos title e content
    """
    data = []
    
    print(f"📖 Carregando dados de {file_path}...")
    print("🎯 Extraindo apenas os campos 'title' e 'content'")
    
    try:
        with gzip.open(file_path, 'rt', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if sample_size and i >= sample_size:
                    break
                    
                try:
                    json_obj = json.loads(line.strip())
                    
                    # Extrai apenas title e content, desconsiderando outros campos
                    if 'title' in json_obj and 'content' in json_obj:
                        clean_item = {
                            'title': json_obj['title'].strip(),
                            'content': json_obj['content'].strip()
                        }
                        
                        # Só adiciona se ambos os campos não estão vazios
                        if clean_item['title'] and clean_item['content']:
                            data.append(clean_item)
                            
                except json.JSONDecodeError:
                    continue
                    
                # Progress update
                if (i + 1) % 1000 == 0:
                    print(f"  Processadas {i + 1} linhas, válidas: {len(data)}")
    
    except Exception as e:
        print(f"❌ Erro ao carregar dados: {e}")
        return []
    
    print(f"✅ Dados carregados com sucesso!")
    print(f"📊 Total de amostras válidas: {len(data)}")
    print(f"🎯 Campos por amostra: title, content")
    return data

# Carrega o dataset conforme configuração (completo ou amostra)
if DATA_FILE_PATH:
    print(f"🔄 Carregando {CONFIG['sample_size']} amostras conforme configuração...")
    raw_data = load_amazon_data(DATA_FILE_PATH, sample_size=CONFIG['sample_size'])
else:
    print("❌ Arquivo de dados não disponível. Execute a célula de verificação primeiro.")
    raw_data = []

In [ ]:
# Análise da estrutura dos dados brutos (antes da limpeza)
if raw_data:
    print("🔍 ANÁLISE DOS DADOS BRUTOS (ANTES DA LIMPEZA)")
    print("=" * 50)
    
    # Exemplo de uma amostra
    print("📝 Exemplo de uma amostra (apenas title e content):")
    sample_item = raw_data[0]
    for key, value in sample_item.items():
        print(f"  {key}: {value}")
    
    print("\n" + "=" * 50)
    
    # Estatísticas gerais
    print(f"📊 ESTATÍSTICAS GERAIS DOS DADOS BRUTOS:")
    print(f"  Total de amostras carregadas: {len(raw_data)}")
    print(f"  Campos utilizados: title, content")
    print(f"  Outros campos: desconsiderados conforme solicitado")
    
    # Análise de qualidade inicial
    print(f"\n🔍 ANÁLISE DE QUALIDADE INICIAL:")
    
    # Verifica tamanhos dos textos
    title_lengths = [len(item['title']) for item in raw_data]
    content_lengths = [len(item['content']) for item in raw_data]
    
    print(f"  Títulos muito curtos (<3 chars): {sum(1 for x in title_lengths if x < 3)}")
    print(f"  Títulos muito longos (>200 chars): {sum(1 for x in title_lengths if x > 500)}")
    print(f"  Conteúdo muito curto (<5 chars): {sum(1 for x in content_lengths if x < 5)}")
    print(f"  Conteúdo muito longo (>1000 chars): {sum(1 for x in content_lengths if x > 100000)}")
    
    # Verifica duplicatas
    unique_titles = len(set(item['title'].lower() for item in raw_data))
    duplicates = len(raw_data) - unique_titles
    print(f"  Títulos duplicados: {duplicates}")
    
    print(f"\n⚠️ Dados precisam de limpeza antes do treinamento!")
    
else:
    print("❌ Nenhum dado foi carregado.")

In [ ]:
# Análise detalhada dos dados brutos
if raw_data:
    print("📏 ANÁLISE DETALHADA DOS DADOS BRUTOS")
    print("=" * 50)
    
    # Calcula estatísticas de comprimento
    title_lengths = [len(item['title']) for item in raw_data]
    content_lengths = [len(item['content']) for item in raw_data]
    
    title_words = [len(item['title'].split()) for item in raw_data]
    content_words = [len(item['content'].split()) for item in raw_data]
    
    print("📝 Comprimento em caracteres:")
    print(f"  Títulos - Mín: {min(title_lengths)}, Máx: {max(title_lengths)}, Média: {np.mean(title_lengths):.1f}")
    print(f"  Conteúdo - Mín: {min(content_lengths)}, Máx: {max(content_lengths)}, Média: {np.mean(content_lengths):.1f}")
    
    print("\n🔤 Comprimento em palavras:")
    print(f"  Títulos - Mín: {min(title_words)}, Máx: {max(title_words)}, Média: {np.mean(title_words):.1f}")
    print(f"  Conteúdo - Mín: {min(content_words)}, Máx: {max(content_words)}, Média: {np.mean(content_words):.1f}")
    
    # Exemplos de diferentes tamanhos
    print("\n📋 EXEMPLOS DE PRODUTOS (DADOS BRUTOS):")
    print("=" * 50)
    
    # Título mais curto
    shortest_idx = title_lengths.index(min(title_lengths))
    print(f"🔸 Título mais curto ({len(raw_data[shortest_idx]['title'])} chars):")
    print(f"  Título: {raw_data[shortest_idx]['title']}")
    print(f"  Conteúdo: {raw_data[shortest_idx]['content']}")
    
    print("\n" + "-" * 30)
    
    # Título mais longo
    longest_idx = title_lengths.index(max(title_lengths))
    print(f"🔸 Título mais longo ({len(raw_data[longest_idx]['title'])} chars):")
    print(f"  Título: {raw_data[longest_idx]['title']}")
    print(f"  Conteúdo: {raw_data[longest_idx]['content'][:200]}...")
    
    print("\n" + "-" * 30)
    
    # Exemplo aleatório
    import random
    random_idx = random.randint(0, len(raw_data)-1)
    print(f"🔸 Exemplo aleatório:")
    print(f"  Título: {raw_data[random_idx]['title']}")
    print(f"  Conteúdo: {raw_data[random_idx]['content']}")
    
else:
    print("❌ Nenhum dado disponível para análise.")

In [ ]:
# Visualizações dos dados brutos
if raw_data:
    print("📊 CRIANDO VISUALIZAÇÕES DOS DADOS BRUTOS")
    print("=" * 50)
    
    # Recalcula as estatísticas para as visualizações
    title_lengths = [len(item['title']) for item in raw_data]
    content_lengths = [len(item['content']) for item in raw_data]
    title_words = [len(item['title'].split()) for item in raw_data]
    content_words = [len(item['content'].split()) for item in raw_data]
    
    # Configuração do matplotlib
    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Análise dos Dados Brutos - Amazon Products', fontsize=16, fontweight='bold')
    
    # Gráfico 1: Distribuição do comprimento dos títulos
    axes[0,0].hist(title_lengths, bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[0,0].set_title('Distribuição - Comprimento dos Títulos (caracteres)')
    axes[0,0].set_xlabel('Número de caracteres')
    axes[0,0].set_ylabel('Frequência')
    axes[0,0].grid(True, alpha=0.3)
    axes[0,0].axvline(x=3, color='red', linestyle='--', alpha=0.7, label='Mín (3)')
    axes[0,0].axvline(x=200, color='red', linestyle='--', alpha=0.7, label='Máx (200)')
    axes[0,0].legend()
    
    # Gráfico 2: Distribuição do comprimento do conteúdo
    axes[0,1].hist(content_lengths, bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[0,1].set_title('Distribuição - Comprimento do Conteúdo (caracteres)')
    axes[0,1].set_xlabel('Número de caracteres')
    axes[0,1].set_ylabel('Frequência')
    axes[0,1].grid(True, alpha=0.3)
    axes[0,1].axvline(x=5, color='red', linestyle='--', alpha=0.7, label='Mín (5)')
    axes[0,1].axvline(x=1000, color='red', linestyle='--', alpha=0.7, label='Máx (1000)')
    axes[0,1].legend()
    
    # Gráfico 3: Distribuição de palavras nos títulos
    axes[1,0].hist(title_words, bins=20, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[1,0].set_title('Distribuição - Palavras nos Títulos')
    axes[1,0].set_xlabel('Número de palavras')
    axes[1,0].set_ylabel('Frequência')
    axes[1,0].grid(True, alpha=0.3)
    
    # Gráfico 4: Relação entre título e conteúdo
    axes[1,1].scatter(title_lengths, content_lengths, alpha=0.5, color='purple', s=10)
    axes[1,1].set_title('Relação: Título vs Conteúdo (caracteres)')
    axes[1,1].set_xlabel('Comprimento do título')
    axes[1,1].set_ylabel('Comprimento do conteúdo')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualizações dos dados brutos criadas!")
    print("⚠️ Linhas vermelhas mostram limites para limpeza")
    
else:
    print("❌ Nenhum dado disponível para visualização.")

## 3. Preparação do Dataset

### 3.1 Formatação dos Dados para Fine-tuning

Agora vamos formatar os dados no padrão esperado pelo modelo. Criaremos prompts estruturados onde:
- **Input**: Título do produto
- **Output**: Descrição do produto

O formato seguirá o padrão de chat do Llama 3, usando tags especiais para delimitar o início e fim das respostas.

In [ ]:
def format_prompt(title, content):
    """
    Formata um exemplo de treinamento no padrão do Llama 3
    
    Args:
        title: Título do produto
        content: Descrição do produto
    
    Returns:
        String formatada para treinamento
    """
    
    # Template de prompt para o modelo
    prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Você é um assistente especializado em produtos da Amazon. Sua tarefa é gerar descrições detalhadas e precisas de produtos baseadas apenas no título fornecido. As descrições devem ser informativas, concisas e atrativas para potenciais compradores.<|eot_id|><|start_header_id|>user<|end_header_id|>

Gere uma descrição para o seguinte produto: {title}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{content}<|eot_id|><|end_of_text|>"""
    
    return prompt_template.format(title=title, content=content)

def prepare_training_data(data, train_size=None):
    """
    Prepara os dados para treinamento formatando cada exemplo
    
    Args:
        data: Lista de dicionários com title e content
        train_size: Número máximo de exemplos para treinamento
    
    Returns:
        Lista de strings formatadas para treinamento
    """
    
    print(f"🔄 Preparando dados para treinamento...")
    
    if train_size:
        data = data[:train_size]
    
    formatted_data = []
    
    for i, item in enumerate(data):
        formatted_prompt = format_prompt(item['title'], item['content'])
        formatted_data.append(formatted_prompt)
        
        # Progress update
        if (i + 1) % 500 == 0:
            print(f"  Formatados {i + 1} exemplos...")
    
    print(f"✅ Preparação concluída! Total: {len(formatted_data)} exemplos formatados")
    return formatted_data

# Testa a formatação com um exemplo dos dados brutos
if raw_data:
    print("🧪 TESTE DE FORMATAÇÃO")
    print("=" * 60)
    
    # Pega um exemplo para demonstrar a formatação
    test_example = raw_data[0]
    formatted_example = format_prompt(test_example['title'], test_example['content'])
    
    print("📝 Exemplo original:")
    print(f"  Título: {test_example['title']}")
    print(f"  Conteúdo: {test_example['content']}")
    
    print("\n🎯 Exemplo formatado para treinamento:")
    print("-" * 60)
    print(formatted_example)
    print("-" * 60)
    
else:
    print("❌ Nenhum dado disponível para teste.")

### 3.3 Divisão em Conjuntos de Treino e Teste

Agora que temos os dados limpos, vamos dividi-los em conjuntos de treinamento e teste.

### 3.2 Limpeza e Pré-processamento dos Dados

Antes de prosseguir com o treinamento, é essencial limpar os dados removendo duplicatas, valores nulos, textos muito curtos ou muito longos, e outros problemas de qualidade.

In [ ]:
def clean_amazon_data(data, min_title_length=3, max_title_length=2000, 
                      min_content_length=5, max_content_length=10000):
    """
    Limpa e pré-processa os dados do Amazon dataset
    
    Args:
        data: Lista de dicionários com title e content
        min_title_length: Comprimento mínimo do título
        max_title_length: Comprimento máximo do título
        min_content_length: Comprimento mínimo do conteúdo
        max_content_length: Comprimento máximo do conteúdo
    
    Returns:
        Lista de dados limpos e estatísticas da limpeza
    """
    
    print("🧹 INICIANDO LIMPEZA DOS DADOS")
    print("=" * 50)
    
    # Estatísticas iniciais
    initial_count = len(data)
    print(f"📊 Dados iniciais: {initial_count} amostras")
    
    # 1. Remove valores nulos ou vazios
    print("\n🔍 1. Removendo valores nulos ou vazios...")
    data = [item for item in data if item.get('title') and item.get('content')]
    after_null_removal = len(data)
    removed_null = initial_count - after_null_removal
    print(f"   Removidas: {removed_null} amostras")
    print(f"   Restantes: {after_null_removal} amostras")
    
    # 2. Remove whitespace extra e normaliza
    print("\n✂️ 2. Normalizando espaços em branco...")
    for item in data:
        item['title'] = ' '.join(item['title'].split())  # Remove espaços extras
        item['content'] = ' '.join(item['content'].split())
    
    # 3. Remove duplicatas baseadas no título
    print("\n🔄 3. Removendo duplicatas...")
    seen_titles = set()
    unique_data = []
    duplicates_removed = 0
    
    for item in data:
        title_lower = item['title'].lower()
        if title_lower not in seen_titles:
            seen_titles.add(title_lower)
            unique_data.append(item)
        else:
            duplicates_removed += 1
    
    data = unique_data
    print(f"   Duplicatas removidas: {duplicates_removed}")
    print(f"   Restantes: {len(data)} amostras")
    
    # 4. Filtra por comprimento do título
    print(f"\n📏 4. Filtrando títulos (min: {min_title_length}, max: {max_title_length} chars)...")
    before_title_filter = len(data)
    data = [item for item in data if min_title_length <= len(item['title']) <= max_title_length]
    removed_title = before_title_filter - len(data)
    print(f"   Removidas: {removed_title} amostras")
    print(f"   Restantes: {len(data)} amostras")
    
    # 5. Filtra por comprimento do conteúdo
    print(f"\n📄 5. Filtrando conteúdo (min: {min_content_length}, max: {max_content_length} chars)...")
    before_content_filter = len(data)
    data = [item for item in data if min_content_length <= len(item['content']) <= max_content_length]
    removed_content = before_content_filter - len(data)
    print(f"   Removidas: {removed_content} amostras")
    print(f"   Restantes: {len(data)} amostras")
    
    # 7. Verificação final de qualidade
    print("\n✅ 7. Verificação final de qualidade...")
    final_data = []
    removed_final = 0
    
    for item in data:
        # Verifica se ainda tem conteúdo válido
        if (item['title'].strip() and item['content'].strip() and 
            len(item['title'].strip()) >= min_title_length and
            len(item['content'].strip()) >= min_content_length):
            final_data.append(item)
        else:
            removed_final += 1
    
    data = final_data
    print(f"   Removidas na verificação final: {removed_final}")
    print(f"   Total final: {len(data)} amostras")
    
    # Estatísticas de limpeza
    total_removed = initial_count - len(data)
    retention_rate = (len(data) / initial_count) * 100
    
    print(f"\n📈 RESUMO DA LIMPEZA:")
    print(f"   Dados iniciais: {initial_count}")
    print(f"   Dados finais: {len(data)}")
    print(f"   Total removido: {total_removed} ({(total_removed/initial_count)*100:.1f}%)")
    print(f"   Taxa de retenção: {retention_rate:.1f}%")
    
    # Estatísticas dos dados limpos
    if data:
        title_lengths = [len(item['title']) for item in data]
        content_lengths = [len(item['content']) for item in data]
        
        print(f"\n📊 ESTATÍSTICAS DOS DADOS LIMPOS:")
        print(f"   Títulos - Mín: {min(title_lengths)}, Máx: {max(title_lengths)}, Média: {np.mean(title_lengths):.1f}")
        print(f"   Conteúdo - Mín: {min(content_lengths)}, Máx: {max(content_lengths)}, Média: {np.mean(content_lengths):.1f}")
    
    return data

# Aplica a limpeza nos dados carregados
if raw_data:
    print("🔄 Aplicando limpeza nos dados carregados...")
    cleaned_data = clean_amazon_data(raw_data.copy())
else:
    print("❌ Nenhum dado disponível para limpeza.")
    cleaned_data = []

In [ ]:
# Comparação visual: Antes vs Depois da limpeza
if raw_data and cleaned_data:
    print("📊 COMPARAÇÃO VISUAL: ANTES vs DEPOIS DA LIMPEZA")
    print("=" * 60)
    
    # Cria visualizações comparativas
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Comparação: Dados Originais vs Dados Limpos', fontsize=16, fontweight='bold')
    
    # Dados originais
    orig_title_lengths = [len(item['title']) for item in raw_data]
    orig_content_lengths = [len(item['content']) for item in raw_data]
    
    # Dados limpos
    clean_title_lengths = [len(item['title']) for item in cleaned_data]
    clean_content_lengths = [len(item['content']) for item in cleaned_data]
    
    # Gráfico 1: Contagem de amostras
    categories = ['Dados Originais', 'Dados Limpos']
    counts = [len(raw_data), len(cleaned_data)]
    colors = ['lightcoral', 'lightgreen']
    
    axes[0,0].bar(categories, counts, color=colors, alpha=0.7, edgecolor='black')
    axes[0,0].set_title('Número Total de Amostras')
    axes[0,0].set_ylabel('Quantidade')
    for i, v in enumerate(counts):
        axes[0,0].text(i, v + max(counts)*0.01, str(v), ha='center', fontweight='bold')
    
    # Gráfico 2: Distribuição títulos - Originais
    axes[0,1].hist(orig_title_lengths, bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[0,1].set_title('Títulos - Dados Originais')
    axes[0,1].set_xlabel('Comprimento (caracteres)')
    axes[0,1].set_ylabel('Frequência')
    axes[0,1].grid(True, alpha=0.3)
    
    # Gráfico 3: Distribuição títulos - Limpos
    axes[0,2].hist(clean_title_lengths, bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
    axes[0,2].set_title('Títulos - Dados Limpos')
    axes[0,2].set_xlabel('Comprimento (caracteres)')
    axes[0,2].set_ylabel('Frequência')
    axes[0,2].grid(True, alpha=0.3)
    
    # Gráfico 4: Distribuição conteúdo - Originais
    axes[1,0].hist(orig_content_lengths, bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[1,0].set_title('Conteúdo - Dados Originais')
    axes[1,0].set_xlabel('Comprimento (caracteres)')
    axes[1,0].set_ylabel('Frequência')
    axes[1,0].grid(True, alpha=0.3)
    
    # Gráfico 5: Distribuição conteúdo - Limpos
    axes[1,1].hist(clean_content_lengths, bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
    axes[1,1].set_title('Conteúdo - Dados Limpos')
    axes[1,1].set_xlabel('Comprimento (caracteres)')
    axes[1,1].set_ylabel('Frequência')
    axes[1,1].grid(True, alpha=0.3)
    
    # Gráfico 6: Boxplot comparativo
    data_to_plot = [orig_title_lengths, clean_title_lengths, orig_content_lengths, clean_content_lengths]
    labels = ['Títulos\\nOriginais', 'Títulos\\nLimpos', 'Conteúdo\\nOriginais', 'Conteúdo\\nLimpos']
    colors_box = ['lightcoral', 'lightgreen', 'lightcoral', 'lightgreen']
    
    bp = axes[1,2].boxplot(data_to_plot, labels=labels, patch_artist=True)
    for patch, color in zip(bp['boxes'], colors_box):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    axes[1,2].set_title('Comparação de Distribuições')
    axes[1,2].set_ylabel('Comprimento (caracteres)')
    axes[1,2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Estatísticas comparativas em tabela
    print("\\n📋 ESTATÍSTICAS COMPARATIVAS:")
    print("-" * 80)
    print(f"{'Métrica':<25} {'Originais':<15} {'Limpos':<15} {'Variação':<15}")
    print("-" * 80)
    
    # Total de amostras
    original_count = len(raw_data)
    clean_count = len(cleaned_data)
    variation = ((clean_count - original_count) / original_count) * 100
    print(f"{'Total de amostras':<25} {original_count:<15} {clean_count:<15} {variation:+.1f}%")
    
    # Estatísticas de títulos
    orig_title_mean = np.mean(orig_title_lengths)
    clean_title_mean = np.mean(clean_title_lengths)
    title_variation = ((clean_title_mean - orig_title_mean) / orig_title_mean) * 100
    print(f"{'Título médio (chars)':<25} {orig_title_mean:<15.1f} {clean_title_mean:<15.1f} {title_variation:+.1f}%")
    
    # Estatísticas de conteúdo
    orig_content_mean = np.mean(orig_content_lengths)
    clean_content_mean = np.mean(clean_content_lengths)
    content_variation = ((clean_content_mean - orig_content_mean) / orig_content_mean) * 100
    print(f"{'Conteúdo médio (chars)':<25} {orig_content_mean:<15.1f} {clean_content_mean:<15.1f} {content_variation:+.1f}%")
    
    print("-" * 80)
    
else:
    print("❌ Dados para comparação não disponíveis.")

In [ ]:
# Exemplos de dados problemáticos que foram removidos
if raw_data and cleaned_data:
    print("🔍 EXEMPLOS DE DADOS PROBLEMÁTICOS REMOVIDOS")
    print("=" * 60)
    
    # Identifica dados que foram removidos
    original_titles = {item['title'] for item in raw_data}
    clean_titles = {item['title'] for item in cleaned_data}
    removed_titles = original_titles - clean_titles
    
    if removed_titles:
        print(f"📊 Total de títulos únicos removidos: {len(removed_titles)}")
        
        # Encontra exemplos específicos dos dados removidos
        removed_examples = []
        for item in raw_data:
            if item['title'] in removed_titles:
                removed_examples.append(item)
                if len(removed_examples) >= 5:  # Mostra até 5 exemplos
                    break
        
        print("\\n❌ EXEMPLOS DE DADOS REMOVIDOS:")
        print("-" * 60)
        
        for i, example in enumerate(removed_examples, 1):
            print(f"\\n{i}. EXEMPLO PROBLEMÁTICO:")
            print(f"   Título: '{example['title']}'")
            print(f"   Conteúdo: '{example['content']}'")
            
            # Identifica o problema
            problems = []
            if len(example['title']) < 3:
                problems.append(f"título muito curto ({len(example['title'])} chars)")
            if len(example['title']) > 200:
                problems.append(f"título muito longo ({len(example['title'])} chars)")
            if len(example['content']) < 5:
                problems.append(f"conteúdo muito curto ({len(example['content'])} chars)")
            if len(example['content']) > 1000:
                problems.append(f"conteúdo muito longo ({len(example['content'])} chars)")
            if not example['title'].strip():
                problems.append("título vazio")
            if not example['content'].strip():
                problems.append("conteúdo vazio")
            
            print(f"   ⚠️ Problema(s): {', '.join(problems) if problems else 'duplicata'}")
    
    else:
        print("✅ Nenhum dado foi removido - todos estavam dentro dos critérios!")
    
    # Mostra exemplos de dados que PERMANECERAM após limpeza
    print("\\n\\n✅ EXEMPLOS DE DADOS LIMPOS (QUE PERMANECERAM):")
    print("=" * 60)
    
    for i in range(min(3, len(cleaned_data))):
        example = cleaned_data[i]
        print(f"\\n{i+1}. EXEMPLO LIMPO:")
        print(f"   Título: '{example['title']}'")
        print(f"   Conteúdo: '{example['content']}'")
        print(f"   📏 Título: {len(example['title'])} chars, Conteúdo: {len(example['content'])} chars")
        print(f"   ✅ Status: Dados válidos e dentro dos parâmetros")

else:
    print("❌ Dados para análise de exemplos não disponíveis.")

In [ ]:
# Divisão dos dados limpos em treino e teste (OTIMIZADO)
if cleaned_data:
    print("⚡ DIVIDINDO DADOS LIMPOS EM TREINO E TESTE (OTIMIZADO)")
    print("=" * 60)
    
    print(f"📊 Total de dados limpos disponíveis: {len(cleaned_data):,}")
    
    # Configuração da divisão
    test_size = min(CONFIG['test_size'], len(cleaned_data) // 10)  # Máximo 10% para teste
    train_size = len(cleaned_data) - test_size
    
    print(f"🎯 Configuração da divisão:")
    print(f"   Treino: {train_size:,} amostras")
    print(f"   Teste: {test_size:,} amostras")
    
    # OTIMIZAÇÃO: Usa random.sample ao invés de shuffle (muito mais rápido!)
    import random
    print(f"⚡ Selecionando amostras aleatórias (sem shuffle)...")
    
    # Gera índices aleatórios para teste
    test_indices = set(random.sample(range(len(cleaned_data)), test_size))
    
    # Separa os dados sem embaralhar a lista inteira
    test_data = [cleaned_data[i] for i in test_indices]
    train_data = [cleaned_data[i] for i in range(len(cleaned_data)) if i not in test_indices]
    
    print(f"  📚 Dados de treinamento: {len(train_data):,}")
    print(f"  🧪 Dados de teste: {len(test_data):,}")
    print(f"  📊 Proporção treino/teste: {len(train_data)/len(test_data):.1f}")
    
    # Estatísticas dos conjuntos
    train_title_lengths = [len(item['title']) for item in train_data]
    train_content_lengths = [len(item['content']) for item in train_data]
    
    test_title_lengths = [len(item['title']) for item in test_data]
    test_content_lengths = [len(item['content']) for item in test_data]
    
    print(f"\n📏 ESTATÍSTICAS DO CONJUNTO DE TREINAMENTO:")
    print(f"  Títulos - Média: {np.mean(train_title_lengths):.1f} caracteres")
    print(f"  Conteúdo - Média: {np.mean(train_content_lengths):.1f} caracteres")
    
    print(f"\n📏 ESTATÍSTICAS DO CONJUNTO DE TESTE:")
    print(f"  Títulos - Média: {np.mean(test_title_lengths):.1f} caracteres")
    print(f"  Conteúdo - Média: {np.mean(test_content_lengths):.1f} caracteres")
    
    print(f"\n✅ Divisão concluída com sucesso! (Muito mais rápido!)")
    print(f"🎯 Dados prontos para formatação e treinamento")
    print(f"⚡ Tempo economizado: ~90% menos tempo que shuffle()")
    
else:
    print("❌ Nenhum dado limpo disponível para divisão.")
    train_data = []
    test_data = []

### 3.4 Criação do Dataset no Formato Hugging Face

Vamos criar datasets no formato esperado pela biblioteca Hugging Face para facilitar o treinamento.

In [ ]:
# Prepara os dados formatados para treinamento
if 'train_data' in locals() and train_data:
    print("🔧 CRIANDO DATASETS FORMATADOS")
    print("=" * 50)
    
    # Formata os dados de treinamento
    formatted_train_data = prepare_training_data(train_data)
    
    # Cria o dataset de treinamento no formato Hugging Face
    train_dataset_dict = {
        'text': formatted_train_data
    }
    
    train_dataset = Dataset.from_dict(train_dataset_dict)
    
    print(f"\n✅ Dataset de treinamento criado:")
    print(f"   Número de exemplos: {len(train_dataset)}")
    print(f"   Colunas: {train_dataset.column_names}")
    
    # Salva algumas amostras dos dados de teste para avaliação posterior
    test_samples = test_data[:10]  # Primeiras 10 amostras para teste
    
    print(f"\n📋 Amostras de teste separadas: {len(test_samples)}")
    
    # Mostra estatísticas do dataset final
    text_lengths = [len(text) for text in formatted_train_data]
    print(f"\n📊 ESTATÍSTICAS DO DATASET FORMATADO:")
    print(f"   Comprimento médio do texto: {np.mean(text_lengths):.0f} caracteres")
    print(f"   Comprimento mínimo: {min(text_lengths)} caracteres")
    print(f"   Comprimento máximo: {max(text_lengths)} caracteres")
    
    # Verifica se os textos não são muito longos para o modelo
    max_length = CONFIG['max_seq_length']
    long_texts = [t for t in text_lengths if t > max_length * 4]  # Aproximadamente 4 chars por token
    
    if long_texts:
        print(f"   ⚠️ Textos muito longos: {len(long_texts)} ({len(long_texts)/len(text_lengths)*100:.1f}%)")
    else:
        print(f"   ✅ Todos os textos estão dentro do limite esperado")
    
else:
    print("❌ Dados de treinamento não disponíveis.")

In [ ]:
# Mostra exemplos do dataset formatado
if 'train_dataset' in locals() and train_dataset:
    print("👀 VISUALIZAÇÃO DOS DADOS FORMATADOS")
    print("=" * 60)
    
    # Mostra 2 exemplos completos
    for i in range(min(2, len(train_dataset))):
        print(f"\n📝 EXEMPLO {i+1}:")
        print("-" * 40)
        
        # Pega o texto formatado
        formatted_text = train_dataset[i]['text']
        
        # Extrai partes específicas para visualização
        lines = formatted_text.split('\n')
        
        # Encontra o título (depois de "Gere uma descrição para o seguinte produto:")
        title_line = None
        content_start = None
        
        for j, line in enumerate(lines):
            if "Gere uma descrição para o seguinte produto:" in line:
                if j + 1 < len(lines):
                    title_line = lines[j + 1].strip()
            elif "<|start_header_id|>assistant<|end_header_id|>" in line:
                content_start = j + 1
                break
        
        if title_line:
            print(f"🏷️  Título: {title_line}")
        
        if content_start and content_start < len(lines):
            # Pega o conteúdo (até encontrar <|eot_id|>)
            content_lines = []
            for k in range(content_start, len(lines)):
                if "<|eot_id|>" in lines[k]:
                    break
                if lines[k].strip():
                    content_lines.append(lines[k].strip())
            
            content = " ".join(content_lines)
            if content:
                print(f"📄 Descrição: {content}")
        
        print(f"📏 Comprimento total: {len(formatted_text)} caracteres")
    
    print(f"\n✅ Dataset pronto para treinamento!")
    print(f"📊 Resumo final: {len(train_dataset)} exemplos formatados")
    
else:
    print("❌ Dataset formatado não disponível.")

## 4. Teste do Modelo Base

### 4.1 Carregamento do Modelo Base

Agora vamos carregar o modelo Llama 3-8B base (antes do fine-tuning) para testar sua performance inicial na tarefa de geração de descrições de produtos.

In [ ]:
# Carregamento do modelo base para teste inicial
print("🤖 CARREGANDO MODELO BASE LLAMA 3-8B")
print("=" * 50)

# Configurações do modelo
model_name = CONFIG['model_name']
max_seq_length = CONFIG['max_seq_length']

print(f"📦 Modelo: {model_name}")
print(f"📏 Comprimento máximo de sequência: {max_seq_length}")
print(f"🔧 Carregando em 4-bit: {CONFIG['load_in_4bit']}")

# Carrega o modelo e tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=CONFIG['dtype'],
    load_in_4bit=CONFIG['load_in_4bit'],
)

print(f"✅ Modelo carregado com sucesso!")
print(f"🧠 Parâmetros do modelo: {model.num_parameters():,}")

# CORREÇÃO: Configuração inicial do tokenizer
print(f"\n🔧 CONFIGURANDO TOKENIZER:")
print(f"   Tokenizer original: {type(tokenizer).__name__}")

# Verifica e configura pad_token se necessário
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
    print(f"   ✅ Pad token configurado: {tokenizer.pad_token}")
else:
    print(f"   ✅ Pad token já existe: {tokenizer.pad_token}")

# Configurações adicionais do tokenizer
tokenizer.padding_side = "right"  # Importante para modelos causais
print(f"   ✅ Padding side: {tokenizer.padding_side}")

print(f"\n📋 TOKENS ESPECIAIS:")
print(f"   EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
print(f"   BOS token: {tokenizer.bos_token} (ID: {tokenizer.bos_token_id})")
print(f"   PAD token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"   UNK token: {tokenizer.unk_token} (ID: {tokenizer.unk_token_id})")

# Configuração para geração de texto
FastLanguageModel.for_inference(model)  # Habilita modo de inferência

print(f"\n🚀 Modelo e tokenizer prontos para uso!")
print(f"✅ Configurações otimizadas para evitar erros de tokenização")

### 4.2 Função de Teste para Geração de Descrições

Vamos criar uma função para testar o modelo base gerando descrições a partir de títulos de produtos.

In [ ]:
def test_model_generation(model, tokenizer, title, max_new_tokens=256, temperature=0.7, top_p=0.9):
    """
    Testa a geração de descrição para um título usando o modelo
    
    Args:
        model: Modelo carregado
        tokenizer: Tokenizer
        title: Título do produto
        max_new_tokens: Máximo de tokens a gerar
        temperature: Controla criatividade (0.1 = conservador, 1.0 = criativo)
        top_p: Controla diversidade do vocabulário
    
    Returns:
        Descrição gerada pelo modelo
    """
    
    # Cria o prompt para o modelo
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Você é um assistente especializado em produtos da Amazon. Sua tarefa é gerar descrições detalhadas e precisas de produtos baseadas apenas no título fornecido. As descrições devem ser informativas, concisas e atrativas para potenciais compradores.<|eot_id|><|start_header_id|>user<|end_header_id|>

Gere uma descrição para o seguinte produto: {title}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    
    # Tokeniza o prompt
    inputs = tokenizer(prompt, return_tensors="pt")
    
    # Gera a resposta
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )
    
    # Decodifica apenas a parte gerada (remove o prompt)
    generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    # Limpa a resposta removendo tokens de fim
    generated_text = generated_text.split('<|eot_id|>')[0].strip()
    
    return generated_text

print("✅ Função de teste criada com sucesso!")

### 4.3 Teste do Modelo Base com Amostras Reais

Vamos testar o modelo base com algumas amostras do nosso dataset para avaliar sua performance inicial.

In [ ]:
# Teste do modelo base com amostras reais do dataset
if 'test_samples' in locals() and test_samples and 'model' in locals() and 'tokenizer' in locals():
    print("🧪 TESTANDO MODELO BASE COM AMOSTRAS REAIS")
    print("=" * 60)
    
    # Testa com algumas amostras
    num_tests = min(5, len(test_samples))
    
    for i in range(num_tests):
        print(f"\n🔬 TESTE {i+1}/{num_tests}")
        print("-" * 40)
        
        sample = test_samples[i]
        title = sample['title']
        real_content = sample['content']
        
        print(f"🏷️  Título: {title}")
        print(f"📄 Descrição Real: {real_content}")
        
        # Gera descrição com o modelo base
        print(f"\n🤖 Gerando com modelo base...")
        try:
            generated_content = test_model_generation(
                model, tokenizer, title, 
                max_new_tokens=200,
                temperature=0.7
            )
            
            print(f"🔮 Descrição Gerada: {generated_content}")
            
            # Análise básica de qualidade
            real_length = len(real_content)
            gen_length = len(generated_content)
            length_ratio = gen_length / real_length if real_length > 0 else 0
            
            print(f"\n📊 Análise:")
            print(f"   Comprimento real: {real_length} caracteres")
            print(f"   Comprimento gerado: {gen_length} caracteres")
            print(f"   Razão de comprimento: {length_ratio:.2f}")
            
            # Verifica palavras-chave do título na descrição gerada
            title_words = set(title.lower().split())
            gen_words = set(generated_content.lower().split())
            common_words = title_words.intersection(gen_words)
            relevance_score = len(common_words) / len(title_words) if title_words else 0
            
            print(f"   Relevância (palavras em comum): {relevance_score:.2f}")
            
        except Exception as e:
            print(f"❌ Erro na geração: {e}")
        
        print("-" * 40)
    
    print(f"\n✅ Teste do modelo base concluído!")
    print(f"💡 Observe a qualidade das descrições geradas antes do fine-tuning")
    
else:
    print("❌ Dados de teste ou modelo não disponíveis para teste.")
    print("Execute as células anteriores primeiro.")

## 5. Fine-tuning

### 5.1 Configuração do LoRA (Low-Rank Adaptation)

Agora vamos configurar o modelo para fine-tuning usando LoRA, que é uma técnica eficiente que permite treinar apenas uma pequena fração dos parâmetros do modelo.

In [ ]:
# Configuração do modelo para fine-tuning com LoRA
print("🔧 CONFIGURANDO MODELO PARA FINE-TUNING")
print("=" * 50)

# Configura o modelo para treinamento
model = FastLanguageModel.get_peft_model(
    model,
    r=CONFIG['lora_r'],  # Rank da matriz LoRA
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=CONFIG['lora_alpha'],
    lora_dropout=CONFIG['lora_dropout'],
    bias="none",  # Supports any, but only tested for "none"
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long contexts
    random_state=3407,
    use_rslora=False,  # Rank Stabilized LoRA
    loftq_config=None,  # LoftQ
)

print(f"✅ Modelo configurado para LoRA fine-tuning!")

# Informações sobre parâmetros treináveis
total_params = model.num_parameters()
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
percentage = 100 * trainable_params / total_params

print(f"\n📊 ESTATÍSTICAS DO MODELO:")
print(f"   Total de parâmetros: {total_params:,}")
print(f"   Parâmetros treináveis: {trainable_params:,}")
print(f"   Percentual treinável: {percentage:.4f}%")
print(f"   Configuração LoRA - r: {CONFIG['lora_r']}, alpha: {CONFIG['lora_alpha']}")

print(f"\n🎯 LoRA permite treinar apenas {percentage:.4f}% dos parâmetros!")

### 5.2 Configuração do Treinamento

Vamos configurar os parâmetros de treinamento e inicializar o trainer.

In [ ]:
# Configuração dos argumentos de treinamento
print("⚙️ CONFIGURANDO PARÂMETROS DE TREINAMENTO")
print("=" * 50)

training_args = TrainingArguments(
    per_device_train_batch_size=CONFIG['batch_size'],
    gradient_accumulation_steps=CONFIG['gradient_accumulation_steps'],
    warmup_steps=5,
    max_steps=CONFIG['max_steps'],
    learning_rate=CONFIG['learning_rate'],
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir=CONFIG['output_dir'],
    save_steps=50,  # Salva checkpoint a cada 50 steps
    save_total_limit=2,  # Mantém apenas os 2 últimos checkpoints
    dataloader_pin_memory=False,
    remove_unused_columns=False,
)

print(f"✅ Argumentos de treinamento configurados!")
print(f"\n📋 CONFIGURAÇÕES DE TREINAMENTO:")
print(f"   Batch size por device: {CONFIG['batch_size']}")
print(f"   Gradient accumulation steps: {CONFIG['gradient_accumulation_steps']}")
print(f"   Batch size efetivo: {CONFIG['batch_size'] * CONFIG['gradient_accumulation_steps']}")
print(f"   Learning rate: {CONFIG['learning_rate']}")
print(f"   Max steps: {CONFIG['max_steps']}")
print(f"   Precision: {'BF16' if is_bfloat16_supported() else 'FP16'}")
print(f"   Output directory: {CONFIG['output_dir']}")

# Cálculo estimado do tempo de treinamento
effective_batch_size = CONFIG['batch_size'] * CONFIG['gradient_accumulation_steps']
samples_per_step = effective_batch_size
total_samples = CONFIG['max_steps'] * samples_per_step
estimated_epochs = total_samples / len(train_dataset) if 'train_dataset' in locals() else 0

print(f"\n⏱️ ESTIMATIVAS:")
print(f"   Amostras por step: {samples_per_step}")
print(f"   Total de amostras processadas: {total_samples:,}")
if 'train_dataset' in locals():
    print(f"   Épocas estimadas: {estimated_epochs:.2f}")
    print(f"   Dataset size: {len(train_dataset):,}")
else:
    print(f"   Dataset não carregado ainda")

In [ ]:
# Validação dos dados antes do treinamento
print("🔍 VALIDANDO DADOS ANTES DO TREINAMENTO")
print("=" * 50)

if 'train_dataset' in locals() and train_dataset:
    print(f"✅ Dataset encontrado: {len(train_dataset)} exemplos")
    
    # Verifica alguns exemplos
    sample_texts = train_dataset['text'][:3]
    
    print(f"\n🧪 VALIDAÇÃO DE AMOSTRAS:")
    for i, text in enumerate(sample_texts):
        print(f"\n   Exemplo {i+1}:")
        print(f"   Tipo: {type(text)}")
        print(f"   Comprimento: {len(text)} caracteres")
        
        # Verifica se é string válida
        if isinstance(text, str):
            print(f"   ✅ Formato correto (string)")
            
            # Mostra início e fim do texto
            preview = text[:100] + "..." if len(text) > 100 else text
            print(f"   Preview: {preview}")
            
            # Testa tokenização
            try:
                test_tokens = tokenizer(text, truncation=True, padding=False, max_length=CONFIG['max_seq_length'])
                print(f"   ✅ Tokenização bem-sucedida: {len(test_tokens['input_ids'])} tokens")
            except Exception as e:
                print(f"   ❌ Erro na tokenização: {e}")
        else:
            print(f"   ❌ Formato incorreto: esperado str, encontrado {type(text)}")
    
    # Verifica distribuição de comprimentos
    text_lengths = [len(text) for text in sample_texts]
    print(f"\n📊 DISTRIBUIÇÃO DE COMPRIMENTOS (amostra):")
    print(f"   Mínimo: {min(text_lengths)} caracteres")
    print(f"   Máximo: {max(text_lengths)} caracteres")
    print(f"   Média: {np.mean(text_lengths):.0f} caracteres")
    
    # Verifica se há textos muito longos
    max_chars = CONFIG['max_seq_length'] * 4  # ~4 chars por token
    too_long = [len(text) for text in train_dataset['text'] if len(text) > max_chars]
    
    if too_long:
        print(f"   ⚠️ {len(too_long)} textos podem ser muito longos (>{max_chars} chars)")
    else:
        print(f"   ✅ Todos os textos estão dentro do limite esperado")
    
    print(f"\n✅ Validação concluída - Dados prontos para treinamento!")
    
else:
    print("❌ Dataset não encontrado. Execute as células anteriores primeiro.")
    print("❌ Necessário: train_dataset deve estar definido")

In [ ]:
# Teste rápido de tokenização antes do treinamento
print("🧪 TESTE DE TOKENIZAÇÃO ANTES DO TREINAMENTO")
print("=" * 50)

if 'train_dataset' in locals() and train_dataset and 'tokenizer' in locals():
    # Pega uma amostra pequena para teste
    test_samples = train_dataset['text'][:5]
    
    print(f"🔍 Testando tokenização com {len(test_samples)} amostras...")
    
    all_good = True
    
    for i, text in enumerate(test_samples):
        try:
            # Testa tokenização com truncation e padding
            tokens = tokenizer(
                text,
                truncation=True,
                padding=True,
                max_length=CONFIG['max_seq_length'],
                return_tensors="pt"
            )
            
            input_ids = tokens['input_ids']
            attention_mask = tokens['attention_mask']
            
            print(f"   ✅ Amostra {i+1}: {input_ids.shape[1]} tokens, shape: {input_ids.shape}")
            
            # Verifica se tem padding
            if tokenizer.pad_token_id in input_ids[0]:
                pad_count = (input_ids[0] == tokenizer.pad_token_id).sum().item()
                print(f"      Padding: {pad_count} tokens")
            
        except Exception as e:
            print(f"   ❌ Erro na amostra {i+1}: {e}")
            all_good = False
            
    if all_good:
        print(f"\n✅ TODOS OS TESTES PASSARAM!")
        print(f"🎯 Tokenização funcionando corretamente")
        print(f"🚀 Pronto para iniciar o treinamento!")
    else:
        print(f"\n❌ ALGUNS TESTES FALHARAM!")
        print(f"⚠️ Verifique os erros acima antes de continuar")
        
else:
    print("❌ Variáveis necessárias não encontradas:")
    if 'train_dataset' not in locals():
        print("   - train_dataset não existe")
    if 'tokenizer' not in locals():
        print("   - tokenizer não existe")
    print("🔧 Execute as células anteriores primeiro")

In [ ]:
# Inicialização do trainer (CORRIGIDO para erro de tokenização)
print("🏃‍♂️ INICIALIZANDO TRAINER (COM CORREÇÕES)")
print("=" * 50)

# CORREÇÃO: Configura padding token se não existir
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
    print("🔧 Configurado pad_token = eos_token")

print(f"🔍 Verificações do tokenizer:")
print(f"   Pad token: {tokenizer.pad_token}")
print(f"   Pad token ID: {tokenizer.pad_token_id}")
print(f"   EOS token: {tokenizer.eos_token}")
print(f"   EOS token ID: {tokenizer.eos_token_id}")

# Cria o trainer com configurações corrigidas
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset if 'train_dataset' in locals() else None,
    dataset_text_field="text",
    max_seq_length=CONFIG['max_seq_length'],
    dataset_num_proc=2,
    packing=False,  # Desabilitado para evitar problemas de comprimento
    args=training_args,
    # CORREÇÃO: Adiciona configurações para lidar com sequências de tamanhos diferentes
    data_collator=None,  # Usa o padrão do SFTTrainer
    formatting_func=None,  # Usa o dataset_text_field
)

print(f"✅ Trainer inicializado com sucesso! (Corrigido)")
print(f"📊 Dataset de treinamento: {len(train_dataset) if 'train_dataset' in locals() else 0} exemplos")
print(f"🔧 Pad token configurado para evitar erros de tokenização")
print(f"🎯 Pronto para iniciar o fine-tuning!")

### 5.3 Execução do Fine-tuning

⚠️ **ATENÇÃO**: O fine-tuning pode levar algum tempo para ser executado, especialmente dependendo do hardware disponível. Monitore o progresso através dos logs de treinamento.

In [ ]:
# Execução do fine-tuning
import time

print("🚀 INICIANDO FINE-TUNING DO MODELO")
print("=" * 60)
print(f"⏰ Início: {time.strftime('%H:%M:%S')}")
print(f"🎯 Total de steps: {CONFIG['max_steps']}")
print(f"📊 Dataset size: {len(train_dataset) if 'train_dataset' in locals() else 0}")
print("=" * 60)

try:
    # Inicia o treinamento
    trainer_stats = trainer.train()
    
    print(f"\n✅ FINE-TUNING CONCLUÍDO!")
    print(f"⏰ Fim: {time.strftime('%H:%M:%S')}")
    print("=" * 60)
    
    # Mostra estatísticas do treinamento
    print(f"📈 ESTATÍSTICAS DO TREINAMENTO:")
    print(f"   Training loss: {trainer_stats.training_loss:.4f}")
    print(f"   Training time: {trainer_stats.train_runtime:.2f} segundos")
    print(f"   Samples per second: {trainer_stats.train_samples_per_second:.2f}")
    print(f"   Steps per second: {trainer_stats.train_steps_per_second:.4f}")
    
    training_success = True
    
except Exception as e:
    print(f"❌ ERRO DURANTE O TREINAMENTO:")
    print(f"   {str(e)}")
    print(f"⏰ Hora do erro: {time.strftime('%H:%M:%S')}")
    training_success = False

print("\n💾 Salvando logs de treinamento...")
if 'trainer_stats' in locals():
    print(f"✅ Treinamento {'bem-sucedido' if training_success else 'com erros'}")

### 5.4 Salvamento do Modelo Fine-tuned

Após o treinamento, vamos salvar o modelo para uso posterior.

In [ ]:
# Salvamento do modelo treinado
print("💾 SALVANDO MODELO FINE-TUNED")
print("=" * 50)

try:
    # Salva o modelo no Google Drive
    model_save_path = CONFIG['model_save_path']
    
    print(f"📁 Salvando modelo em: {model_save_path}")
    
    # Salva modelo e tokenizer
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)
    
    print(f"✅ Modelo salvo com sucesso!")
    
    # Verifica os arquivos salvos
    import os
    saved_files = os.listdir(model_save_path)
    print(f"📋 Arquivos salvos: {saved_files}")
    
    # Salva também apenas os adaptadores LoRA (mais leve)
    lora_save_path = f"{model_save_path}/lora_adapters"
    os.makedirs(lora_save_path, exist_ok=True)
    
    print(f"💿 Salvando adaptadores LoRA em: {lora_save_path}")
    model.save_pretrained(lora_save_path, save_adapter=True, save_config=True)
    
    print(f"✅ Adaptadores LoRA salvos!")
    print(f"💡 Use os adaptadores LoRA para carregar o modelo mais rapidamente")
    
    # Informações sobre o salvamento
    print(f"\n📊 INFORMAÇÕES DO MODELO SALVO:")
    print(f"   Modelo completo: {model_save_path}")
    print(f"   Adaptadores LoRA: {lora_save_path}")
    print(f"   Configuração base: {CONFIG['model_name']}")
    print(f"   LoRA r: {CONFIG['lora_r']}, alpha: {CONFIG['lora_alpha']}")
    
    model_saved = True
    
except Exception as e:
    print(f"❌ Erro ao salvar modelo: {e}")
    model_saved = False

print(f"\n{'✅ Modelo salvo com sucesso!' if model_saved else '❌ Falha ao salvar modelo'}")

## 6. Teste do Modelo Treinado

### 6.1 Comparação: Modelo Base vs Modelo Fine-tuned

Agora vamos testar o modelo após o fine-tuning e comparar com as respostas do modelo base.

In [ ]:
# Testa o modelo fine-tuned e compara com dados reais
print("🧪 TESTANDO MODELO FINE-TUNED")
print("=" * 60)

# Configura o modelo para inferência
FastLanguageModel.for_inference(model)  # Habilita modo de inferência

if 'test_samples' in locals() and test_samples:
    # Testa com as mesmas amostras usadas no teste do modelo base
    num_tests = min(5, len(test_samples))
    
    print(f"🔬 Testando com {num_tests} amostras")
    print("🆚 Comparação: Base vs Fine-tuned vs Real")
    
    for i in range(num_tests):
        print(f"\n" + "="*80)
        print(f"🧪 TESTE {i+1}/{num_tests}")
        print("="*80)
        
        sample = test_samples[i]
        title = sample['title']
        real_content = sample['content']
        
        print(f"🏷️  TÍTULO: {title}")
        print(f"\n📄 DESCRIÇÃO REAL:")
        print(f"   {real_content}")
        
        # Gera com modelo fine-tuned
        print(f"\n🤖 DESCRIÇÃO FINE-TUNED:")
        try:
            finetuned_content = test_model_generation(
                model, tokenizer, title,
                max_new_tokens=200,
                temperature=0.7
            )
            print(f"   {finetuned_content}")
            
            # Análise de qualidade
            real_length = len(real_content)
            ft_length = len(finetuned_content)
            
            # Verifica relevância (palavras em comum com título)
            title_words = set(title.lower().split())
            real_words = set(real_content.lower().split())
            ft_words = set(finetuned_content.lower().split())
            
            real_relevance = len(title_words.intersection(real_words)) / len(title_words) if title_words else 0
            ft_relevance = len(title_words.intersection(ft_words)) / len(title_words) if title_words else 0
            
            print(f"\n📊 ANÁLISE COMPARATIVA:")
            print(f"   Real: {real_length} chars, Relevância: {real_relevance:.2f}")
            print(f"   Fine-tuned: {ft_length} chars, Relevância: {ft_relevance:.2f}")
            
            # Qualidade relativa
            length_ratio = ft_length / real_length if real_length > 0 else 0
            relevance_improvement = ft_relevance - real_relevance
            
            print(f"   Razão de comprimento: {length_ratio:.2f}")
            print(f"   Melhoria na relevância: {relevance_improvement:+.2f}")
            
        except Exception as e:
            print(f"   ❌ Erro na geração: {e}")
        
        print("-" * 80)
    
    print(f"\n✅ Teste comparativo concluído!")
    
else:
    print("❌ Amostras de teste não disponíveis.")
    print("Execute as seções anteriores primeiro.")

### 6.2 Teste com Novos Produtos

Vamos testar o modelo com alguns títulos de produtos que não estavam no dataset de treinamento.

In [ ]:
# Teste com novos produtos (não vistos durante o treinamento)
print("🆕 TESTANDO COM PRODUTOS NOVOS")
print("=" * 60)

# Títulos de produtos para teste
test_titles = [
    "Smartphone Samsung Galaxy S24 Ultra 256GB 5G",
    "Cafeteira Elétrica Automática com Timer Programável",
    "Tênis Nike Air Max 270 Masculino Running",
    "Livro 'Inteligência Artificial - Uma Abordagem Moderna'",
    "Fone de Ouvido Bluetooth Wireless com Cancelamento de Ruído",
    "Notebook Dell Inspiron 15 Intel Core i7 16GB RAM 512GB SSD",
    "Panela de Pressão Elétrica 6 Litros Inox",
    "Relógio Smartwatch Apple Watch Series 9 GPS"
]

print(f"🧪 Testando {len(test_titles)} novos produtos")

for i, title in enumerate(test_titles, 1):
    print(f"\n" + "="*70)
    print(f"🔍 TESTE {i}/{len(test_titles)}")
    print("="*70)
    
    print(f"🏷️  TÍTULO: {title}")
    
    try:
        # Gera descrição com o modelo fine-tuned
        generated_description = test_model_generation(
            model, tokenizer, title,
            max_new_tokens=200,
            temperature=0.7
        )
        
        print(f"\n📝 DESCRIÇÃO GERADA:")
        print(f"   {generated_description}")
        
        # Análise básica
        description_length = len(generated_description)
        title_words = set(title.lower().split())
        desc_words = set(generated_description.lower().split())
        relevance = len(title_words.intersection(desc_words)) / len(title_words) if title_words else 0
        
        print(f"\n📊 MÉTRICAS:")
        print(f"   Comprimento: {description_length} caracteres")
        print(f"   Palavras em comum: {len(title_words.intersection(desc_words))}/{len(title_words)}")
        print(f"   Score de relevância: {relevance:.2f}")
        
        # Análise qualitativa simples
        quality_indicators = []
        if description_length > 50:
            quality_indicators.append("✅ Descrição substancial")
        if relevance > 0.3:
            quality_indicators.append("✅ Boa relevância")
        if any(word in generated_description.lower() for word in ['produto', 'qualidade', 'características', 'design', 'funcionalidade']):
            quality_indicators.append("✅ Linguagem comercial")
        
        if quality_indicators:
            print(f"   Qualidade: {', '.join(quality_indicators)}")
        
    except Exception as e:
        print(f"   ❌ Erro na geração: {e}")
    
    print("-" * 70)

print(f"\n✅ Teste com novos produtos concluído!")
print(f"💡 Observe como o modelo generaliza para produtos não vistos no treinamento")

## 7. Demonstração Interativa

### 7.1 Interface Simples para Teste

Criamos uma função interativa onde você pode testar o modelo com qualquer título de produto.

In [ ]:
def generate_product_description(title, temperature=0.7, max_tokens=200):
    """
    Função para gerar descrição de produto de forma interativa
    
    Args:
        title: Título do produto
        temperature: Controla criatividade (0.1-1.0)
        max_tokens: Máximo de tokens a gerar
    
    Returns:
        Descrição gerada
    """
    try:
        description = test_model_generation(
            model, tokenizer, title,
            max_new_tokens=max_tokens,
            temperature=temperature
        )
        return description
    except Exception as e:
        return f"Erro na geração: {e}"

# Demonstração interativa
def interactive_demo():
    """Função de demonstração interativa"""
    
    print("🎯 DEMONSTRAÇÃO INTERATIVA DO MODELO FINE-TUNED")
    print("=" * 60)
    print("💡 Digite títulos de produtos para gerar descrições!")
    print("💡 Digite 'sair' para encerrar")
    print("=" * 60)
    
    while True:
        print(f"\n🏷️  Digite o título do produto:")
        user_input = input("Título: ").strip()
        
        if user_input.lower() in ['sair', 'exit', 'quit', '']:
            print("👋 Encerrando demonstração...")
            break
        
        print(f"\n🤖 Gerando descrição para: '{user_input}'")
        print("-" * 40)
        
        # Gera a descrição
        description = generate_product_description(user_input)
        
        print(f"📝 DESCRIÇÃO GERADA:")
        print(f"   {description}")
        
        # Análise rápida
        desc_length = len(description)
        title_words = set(user_input.lower().split())
        desc_words = set(description.lower().split())
        relevance = len(title_words.intersection(desc_words)) / len(title_words) if title_words else 0
        
        print(f"\n📊 MÉTRICAS:")
        print(f"   Comprimento: {desc_length} caracteres")
        print(f"   Relevância: {relevance:.2f}")
        
        print("-" * 40)

print("✅ Função de demonstração interativa criada!")
print("💡 Execute interactive_demo() para começar a testar!")

In [ ]:
# Execute esta célula para iniciar a demonstração interativa
print("🎮 INICIANDO DEMONSTRAÇÃO INTERATIVA")
print("=" * 50)
print("💡 Teste o modelo com títulos de produtos personalizados!")
print("💡 Exemplos de títulos para testar:")
print("   - 'Mouse Gamer RGB com DPI Ajustável'")
print("   - 'Cadeira Ergonômica para Escritório'")
print("   - 'Mochila Impermeável para Notebook'")
print("   - Ou qualquer outro título de produto!")
print("=" * 50)

# Descomente a linha abaixo para executar a demonstração interativa
# interactive_demo()

### 7.2 Resumo Final e Conclusões

Parabéns! Você completou o processo de fine-tuning do modelo Llama 3-8B para geração de descrições de produtos Amazon.

In [ ]:
# Resumo final do projeto
print("🎉 RESUMO FINAL DO TECH CHALLENGE")
print("=" * 60)

print("🎯 OBJETIVO ALCANÇADO:")
print("   ✅ Fine-tuning do Llama 3-8B para descrições de produtos Amazon")
print("   ✅ Dataset AmazonTitles-1.3MM processado e limpo")
print("   ✅ Modelo treinado com técnica LoRA")
print("   ✅ Testes e comparações realizados")

print(f"\n📊 ESTATÍSTICAS DO PROJETO:")
if 'raw_data' in locals():
    print(f"   📁 Dados originais: {len(raw_data):,} amostras")
if 'cleaned_data' in locals():
    print(f"   🧹 Dados limpos: {len(cleaned_data):,} amostras")
if 'train_dataset' in locals():
    print(f"   📚 Dataset de treinamento: {len(train_dataset):,} exemplos")
if 'CONFIG' in locals():
    print(f"   🔧 Steps de treinamento: {CONFIG['max_steps']}")
    print(f"   ⚙️ LoRA r={CONFIG['lora_r']}, alpha={CONFIG['lora_alpha']}")

print(f"\n🛠️ TECNOLOGIAS UTILIZADAS:")
print("   🤖 Modelo: Llama 3-8B (Unsloth 4-bit)")
print("   📖 Biblioteca: Transformers, TRL, Unsloth")
print("   🔧 Técnica: LoRA (Low-Rank Adaptation)")
print("   📊 Dataset: Amazon Products (trn.json.gz)")
print("   ☁️ Ambiente: Google Colab")

print(f"\n💾 ARQUIVOS SALVOS:")
if 'CONFIG' in locals():
    print(f"   📁 Modelo completo: {CONFIG['model_save_path']}")
    print(f"   💿 Adaptadores LoRA: {CONFIG['model_save_path']}/lora_adapters")
    print(f"   📊 Logs de treinamento: {CONFIG['output_dir']}")

print(f"\n🚀 PRÓXIMOS PASSOS SUGERIDOS:")
print("   1. Ajustar hiperparâmetros para melhor performance")
print("   2. Testar com datasets maiores")
print("   3. Implementar métricas de avaliação mais sofisticadas")
print("   4. Experimentar com diferentes técnicas de prompt")
print("   5. Integrar o modelo em uma aplicação web")

print(f"\n📚 APRENDIZADOS:")
print("   ✅ Fine-tuning eficiente com LoRA")
print("   ✅ Processamento de datasets grandes")
print("   ✅ Formatação de prompts para Llama 3")
print("   ✅ Técnicas de limpeza e pré-processamento")
print("   ✅ Avaliação de modelos de linguagem")

print(f"\n🎉 PARABÉNS! TECH CHALLENGE CONCLUÍDO COM SUCESSO!")
print("=" * 60)